In [1]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# prompt: make this pwd /content/drive/MyDrive/New_pheonix

%cd /content/drive/MyDrive/New_pheonix


/content/drive/MyDrive/New_pheonix


In [3]:
import argparse
import torch
from torch import nn, optim
from torch.optim import lr_scheduler
from tqdm import tqdm
import time
import random
import numpy as np
import os
import TimeLLM
from TimeSeries import data_provider
from tools import EarlyStopping, adjust_learning_rate, vali, load_content
import sys
import warnings
warnings.filterwarnings("ignore")
%load_ext tensorboard

In [4]:
fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)


In [5]:
args = argparse.Namespace(
    root_path='./',  # Assuming the file is in the current directory
    data_path='NVDA.csv',  # Your CSV file
    seq_len=384,
    label_len=96,
    pred_len=96,
    features='M',  # 'M' for multivariate, 'S' for univariate
    target=None,  # Replace with the actual name of the target column in your CSV
    embed='timeF',  # Use 'timeF' for time feature encoding
    scale=True,
    percent=100,
    num_workers=0,
    batch_size=2,
    freq='d',
    model_id='test',
    model_comment='none',
    model='TimeLLM',
    seed=2021,
    checkpoints='/content/drive/MyDrive/New_pheonix/checkpoints',
    d_model=512,
    n_heads=8,
    e_layers=2,
    d_layers=1,
    d_ff=4096,
    dropout=0.1,
    activation='gelu',
    eval_batch_size=1,
    patience=10,
    learning_rate=0.0001,
    train_epochs=1,
    loss='MSE',
    lradj='type1',
    use_amp=False,
    llm_layers=12,
    task_name = 'long_term_forecast',
    llm_model = 'LLAMA',
    llm_dim = 4096,
    enc_in = 7,
    patch_len = 16 ,
    stride = 8,
    prompt_domain = False,
    content = None,
    output_attention=False

)


In [6]:
model = TimeLLM.Model(args).float()

config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.28k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message


In [7]:
train_data, train_loader = data_provider(args, 'train')
vali_data, vali_loader = data_provider(args, 'val')
test_data, test_loader = data_provider(args, 'test')

data_x shape: (870, 6)
data_y shape: (870, 6)
data_x shape: (509, 6)
data_y shape: (509, 6)
data_x shape: (632, 6)
data_y shape: (632, 6)


In [8]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Model(
  (llm_model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-11): 12 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (d

In [9]:
torch.cuda.empty_cache()

In [10]:
optimizer = optim.Adam(model.parameters(), lr=args.learning_rate)
criterion = nn.MSELoss()
mae_metric = nn.L1Loss()
scheduler = lr_scheduler.OneCycleLR(optimizer, max_lr=args.learning_rate, epochs=args.train_epochs, steps_per_epoch=len(train_loader))


In [11]:
if args.use_amp:
    scaler = torch.cuda.amp.GradScaler()

In [12]:
import tensorflow as tf

In [13]:
from torch.utils.tensorboard import SummaryWriter

# Initialize TensorBoard writer
writer = SummaryWriter(log_dir=args.checkpoints)

In [33]:
from torch.utils.tensorboard import SummaryWriter

# Initialize TensorBoard writer
writer = SummaryWriter(log_dir=args.checkpoints)
for epoch in range(args.train_epochs):
    model.train()
    train_loss = []
    epoch_start_time = time.time()

    for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(tqdm(train_loader)):
        optimizer.zero_grad()
        batch_x, batch_y = batch_x.float().to(device), batch_y.float().to(device)
        batch_x_mark, batch_y_mark = batch_x_mark.float().to(device), batch_y_mark.float().to(device)

        # Decoder input
        dec_inp = torch.zeros_like(batch_y[:, -args.pred_len:, :]).float()
        dec_inp = torch.cat([batch_y[:, :args.label_len, :], dec_inp], dim=1).to(device)

        if args.use_amp:
            with torch.cuda.amp.autocast():
                outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
                loss = criterion(outputs, batch_y[:, -args.pred_len:, :])
        else:
            outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
            loss = criterion(outputs, batch_y[:, -args.pred_len:, :])

        if args.use_amp:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()

        train_loss.append(loss.item())

    scheduler.step()

    # Validation and Early Stopping
    train_loss = np.mean(train_loss)
    vali_loss, _ = vali(args, model, vali_data, vali_loader, criterion, mae_metric)
    writer.add_scalar('Loss/train', train_loss, epoch)
    writer.add_scalar('Loss/vali', vali_loss, epoch)
    #early_stopping(vali_loss, model, path=args.checkpoints)

    print(f"Epoch: {epoch+1}, Train Loss: {train_loss:.6f}, Vali Loss: {vali_loss:.6f}, Time: {time.time()-epoch_start_time:.2f}s")

    # if early_stopping.early_stop:
    #     print("Early stopping triggered")
    #     break

torch.save(model.state_dict(), f"{args.checkpoints}/model_epoch_final.pth")


100%|██████████| 1173/1173 [08:34<00:00,  2.28it/s]
90it [00:17,  5.18it/s]


Epoch: 1, Train Loss: 1.338715, Vali Loss: 3.615289, Time: 532.19s


In [14]:
model.load_state_dict(torch.load("/content/drive/MyDrive/New_pheonix/checkpoints/model_epoch_final.pth"))
test_loss, _ = vali(args, model, test_data, test_loader, criterion, mae_metric)
print(f"Test Loss: {test_loss:.6f}")
writer.add_scalar('Loss/Test', test_loss, epoch)

# Close the TensorBoard writer
writer.close()

0it [00:00, ?it/s]We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
LlamaModel is using LlamaSdpaAttention, but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.
459it [06:21,  1.20it/s]


Test Loss: 25.974473


NameError: name 'epoch' is not defined

In [15]:
model.eval()

# Containers to store predictions and actual values
predictions = []
actuals = []

# Disable gradient calculation for evaluation
with torch.no_grad():
    for batch_x, batch_y, batch_x_mark, batch_y_mark in test_loader:
        batch_x, batch_y = batch_x.float().to(device), batch_y.float().to(device)
        batch_x_mark, batch_y_mark = batch_x_mark.float().to(device), batch_y_mark.float().to(device)

        # Prepare decoder input (based on your model's requirements)
        dec_inp = torch.zeros_like(batch_y[:, -args.pred_len:, :]).float()
        dec_inp = torch.cat([batch_y[:, :args.label_len, :], dec_inp], dim=1).to(device)

        # Generate predictions
        outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)

        # Store the predictions and actual values
        predictions.append(outputs.cpu().numpy())
        actuals.append(batch_y[:, -args.pred_len:, :].cpu().numpy())

# Concatenate all batches into a single array
predictions = np.concatenate(predictions, axis=0)
actuals = np.concatenate(actuals, axis=0)


In [19]:
actuals.shape, predictions.shape

((918, 96, 6), (918, 96, 6))

In [20]:
import matplotlib.pyplot as plt

# Select a sample index (e.g., the first sample)
sample_index = 0

# Select a feature index (e.g., the first feature)
feature_index = 0

# Extract the actuals and predictions for the selected sample and feature
actual = actuals[sample_index, :, feature_index]
predicted = predictions[sample_index, :, feature_index]

# Plot the actuals vs. predictions
plt.figure(figsize=(10, 6))
plt.plot(range(96), actual, label='Actual', color='blue')
plt.plot(range(96), predicted, label='Predicted', color='red')
plt.xlabel('Time Step')
plt.ylabel('Value')
plt.title(f'Actual vs Predicted for Sample {sample_index + 1}, Feature {feature_index + 1}')
plt.legend()
plt.show()


In [22]:
import matplotlib.pyplot as plt

# Select a sample index (e.g., the first sample)
sample_index = 0

# Number of features (assuming 6 based on your shape)
num_features = 6

# Create a figure and a set of subplots
plt.figure(figsize=(15, 20))

for feature_index in range(num_features):
    plt.subplot(num_features, 1, feature_index + 1)  # Create a subplot for each feature
    actual = actuals[sample_index, :, feature_index]
    predicted = predictions[sample_index, :, feature_index]

    plt.plot(range(96), actual, label='Actual', color='blue')
    plt.plot(range(96), predicted, label='Predicted', color='red', linestyle='--')
    plt.xlabel('Time Step')
    plt.ylabel(f'Feature {feature_index + 1} Value')
    plt.title(f'Actual vs Predicted for Feature {feature_index + 1}')
    plt.legend()

# Adjust the layout to prevent overlap
plt.tight_layout()
plt.savefig('tets_2.png')
plt.show()


In [21]:
actuals

array([[ 4.3936443 ],
       [ 4.4120135 ],
       [ 4.3744    ],
       ...,
       [-0.7973932 ],
       [-1.0303499 ],
       [ 0.19851312]], dtype=float32)

In [26]:
import matplotlib.pyplot as plt

# Assuming both predictions and actuals are 2D: (num_samples, 1)
# Flatten the arrays to 1D
predictions = predictions.flatten()
actuals = actuals.flatten()

# Plotting predictions vs actuals
plt.figure(figsize=(10, 6))
plt.plot(actuals, label='Actual')
plt.plot(predictions, label='Prediction')

plt.title('Predictions vs Actuals')
plt.xlabel('Sample Index')
plt.ylabel('Value')

plt.legend()
plt.savefig('predictions_vs_actuals.png')
plt.show()

In [24]:
# prompt: save the graph

import matplotlib.pyplot as plt
plt.savefig('predictions_vs_actuals.png')


array([1.8247762 , 2.1348639 , 2.0155523 , ..., 0.5837703 , 0.1800545 ,
       0.07502863], dtype=float32)

In [28]:
# prompt: chose to display 100 datapoins

import matplotlib.pyplot as plt
# Assuming both predictions and actuals are 2D: (number of samples, time steps)
# Adjust indexing based on actual shape
plt.figure(figsize=(10, 6))
plt.plot(actuals[:100], label='Actual')  # Display first 100 data points
plt.plot(predictions[:100], label='Prediction')  # Display first 100 data points

plt.title('Predictions vs Actuals (First 100 Data Points)')
plt.xlabel('Time Step')
plt.ylabel('Value')
plt.savefig('predictions_vs_actuals.png')
plt.legend()
plt.show()
